# Chapter 3

## Figure 3.03 Michaelis-Menten kinetics

In [ ]:
using OrdinaryDiffEq
using ComponentArrays
using UnPack
using Plots
Plots.default(linewidth=2)

In [ ]:
function fullmodel!(D, u, p, t)
    @unpack k1, km1, k2, et = p
    @unpack s, es, p = u
    e = et - es
    v1 = k1 * s * e - km1 * es
    v2 = k2 * es
    D.s = -v1
    D.es = v1 - v2
    D.p = v2
end

In [ ]:
u0 = ComponentArray(s = 5., es = 0., p = 0.)
ps = ComponentArray(k1 = 30., km1 = 1., k2 = 10., et = 1.)
tend = 1.0

prob = ODEProblem(fullmodel!, u0, tend, ps)
sol = solve(prob, Tsit5())

plot(sol, xlabel="Time (AU)", ylabel="Concentration (AU)", legend=:right)

In [ ]:
# Convenience functions
hill(x, k) = x / (x + k)
hill(x, k, n) = hill(x^n, k^n)

function mmmodel(u, p, t)
    @unpack k1, km1, k2, et = p
    return - k2 * et * hill(u, (km1 + k2) / k1)
end

u0simp = sum(u0)
probsimp = ODEProblem(mmmodel, u0simp, tend, ps)
solsimp = solve(probsimp, Tsit5())

pl2 = plot(sol, idxs = [:s, :p], line=(:dash), label=["S (full)" "P (full)"])
pl2 = plot!(pl2, [(t -> solsimp(t)) (t -> u0simp - solsimp(t))], 0.0, tend, label=["S (MM)" "P (MM)"])
pl2 = plot!(pl2, title="Fig. 3.03", xlabel="Time (AU)", ylabel="Concentration (AU)", xlims=(0.0, tend), ylims=(0.0, u0simp), legend=:right)

## Runtime information

In [ ]:
versioninfo()

In [ ]:
using Pkg
Pkg.status()